# Проект: вариант 2
### Задание 2. SQL
### 2.1 Очень усердные ученики.

2.1.1 Условие

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.

2.1.2 Задача

Дана таблица default.peas.

Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.NB! Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

### Решение задачи 2.1

In [2]:
#импортируем нужные библиотеки
import pandahouse as ph
import pandas as pd
import numpy as np

In [3]:
#объявляем параметры подключения
connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

In [4]:
#пишем запрос, и получаем данные из clickhouse в pandas dataframe
query = """
WITH
    --расчет начала текущего месяца (текущим месяцем буду считать последний имеющийся в данных месяц)
    (SELECT toStartOfMonth(MAX(timest)) FROM default.peas) AS beginning_current_month
    
SELECT
    COUNT(st_id) AS diligent_students
FROM
    (
    SELECT
        st_id,
        SUM(correct) AS correct_peas -- посчитаем кол-во правильно решенных заданий
    FROM default.peas
    WHERE
        timest >= beginning_current_month -- отберем события только текущего месяца
    GROUP BY
        st_id
    HAVING
        correct_peas >= 20 -- критерий усердности ученика - решить правильно 20 или более заданий
    )
"""
diligent_students = ph.read_clickhouse(query, connection=connection_default)
diligent_students

,diligent_students
0,136


### 2.2 Оптимизация воронки

2.2.1 Условие

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

2.2.2 Задача

Даны таблицы: default.peas (см. выше), default.studs и default.final_project_check.

Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:

- ARPU 
- ARPAU 
- CR в покупку 
- СR активного пользователя в покупку 
- CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике

ARPU считается относительно всех пользователей, попавших в группы.

Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах.

Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике.

### Решение задачи 2.2

Сначала объединим все данные, необходимые для расчета метрик, в одну таблицу.

Для этого сделаем несколько последовательных джойнов: 
возьмем полный список уникальных id студентов и групп, к которому будем джойнить(default.studs),
лефт джойном добавим к нему все остальные необходимые данные.

In [5]:
query = """
SELECT *
FROM
    (
    SELECT
        A.test_grp AS test_grp,
        A.st_id AS all_students,
        B.st_id AS active_students,
        C.st_id AS active_students_math,
        D.st_id AS paying_students,
        E.st_id AS paying_students_math,
        D.money AS money,
        E.money_math AS money_math --справочно
    FROM
        default.studs AS A -- полный список уникальных id студентов и групп, к которому будем джойнить

        
    LEFT JOIN -- добавляем активных студентов
        (
        SELECT
            st_id,
            SUM(correct) AS correct_peas
        FROM
            default.peas
        GROUP BY
            st_id
        HAVING
            correct_peas > 10 
        ) AS B
    ON
        A.st_id = B.st_id


    LEFT JOIN -- добавляем активных студентов по математике
        (
        SELECT
            st_id,
            SUM(correct) AS correct_peas_math
        FROM
            default.peas
        WHERE
            subject == 'Math'
        GROUP BY
            st_id
        HAVING
            correct_peas_math >= 2 
        ) AS C
    ON
        A.st_id = C.st_id
        
    
    LEFT JOIN -- добавляем платящих студентов и суммы оплаты
        (
        SELECT
            st_id,
            SUM(money) as money
        FROM
            default.final_project_check
        GROUP BY
            st_id
        ) AS D
    ON
        A.st_id = D.st_id
        
        
    LEFT JOIN -- добавляем платящих студентов и суммы оплаты по математике
        (
        SELECT
            st_id,
            SUM(money) AS money_math
        FROM
            default.final_project_check
        WHERE
            subject == 'Math'
        GROUP BY
            st_id
        ) AS E
    ON
        A.st_id = E.st_id
    )
"""
Data_for_metrics = ph.read_clickhouse(query, connection=connection_default)

Data_for_metrics = Data_for_metrics.replace ( '' , np.nan ).replace ( 0 , np.nan )

Data_for_metrics.head()

,test_grp,all_students,active_students,active_students_math,paying_students,paying_students_math,money,money_math
0,pilot,100379,100379,NaN,NaN,NaN,NaN,NaN
1,control,101432,101432,NaN,101432,101432,150000.0,85000.0
2,pilot,104818,NaN,NaN,NaN,NaN,NaN,NaN
3,pilot,104885,NaN,NaN,104885,NaN,140000.0,NaN
4,pilot,104966,104966,NaN,NaN,NaN,NaN,NaN


In [6]:
Data_for_metrics.nunique()

test_grp                  2
all_students            600
active_students         223
active_students_math     91
paying_students          47
paying_students_math     24
money                    12
money_math                1
dtype: int64

In [7]:
Data_for_metrics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   test_grp              600 non-null    object 
 1   all_students          600 non-null    object 
 2   active_students       223 non-null    object 
 3   active_students_math  91 non-null     object 
 4   paying_students       47 non-null     object 
 5   paying_students_math  24 non-null     object 
 6   money                 47 non-null     float64
 7   money_math            24 non-null     float64
dtypes: float64(2), object(6)
memory usage: 37.6+ KB


Теперь на основании полученных выше данных можно рассчитать все метрики.

### Решение задачи 2.2 в одном запросе:

In [8]:
query = """
SELECT
    test_grp,
    ROUND(SUM(money)/COUNT(all_students)) AS ARPU,
    ROUND(SUM(money)/COUNT(active_students)) AS ARPAU,
    ROUND(SUM(money)/COUNT(paying_students)) AS ARPPU, -- этой метрики нет в задаче, но её посмотреть тоже интересно
    ROUND(COUNT(paying_students)/COUNT(all_students),2) AS CR_all_to_paying,
    ROUND(COUNT(paying_students)/COUNT(active_students),2) AS CR_active_to_paying,
    ROUND(COUNT(paying_students_math)/COUNT(active_students_math),2) AS CR_active_to_paying_math
FROM
    (
    SELECT
        A.test_grp AS test_grp,
        A.st_id AS all_students,
        NULLIF(B.st_id, '') AS active_students,     -- превратим пустые строки в NULL, чтобы они не мешали считать кол-во
        NULLIF(C.st_id, '') AS active_students_math,
        NULLIF(D.st_id, '') AS paying_students,
        NULLIF(E.st_id, '') AS paying_students_math,
        D.money AS money,
        E.money_math AS money_math -- справочно
    FROM
        default.studs AS A -- полный список уникальных id студентов и групп, к которому будем джойнить

        
    LEFT JOIN -- добавляем активных студентов
        (
        SELECT
            st_id,
            SUM(correct) AS correct_peas
        FROM
            default.peas
        GROUP BY
            st_id
        HAVING
            correct_peas > 10 
        ) AS B
    ON
        A.st_id = B.st_id


    LEFT JOIN -- добавляем активных студентов по математике
        (
        SELECT
            st_id,
            SUM(correct) AS correct_peas_math
        FROM
            default.peas
        WHERE
            subject == 'Math'
        GROUP BY
            st_id
        HAVING
            correct_peas_math >= 2 
        ) AS C
    ON
        A.st_id = C.st_id
        
    
    LEFT JOIN -- добавляем платящих студентов и суммы оплаты
        (
        SELECT
            st_id,
            SUM(money) as money
        FROM
            default.final_project_check
        GROUP BY
            st_id
        ) AS D
    ON
        A.st_id = D.st_id
        
        
    LEFT JOIN -- добавляем платящих студентов и суммы оплаты по математике
        (
        SELECT
            st_id,
            SUM(money) AS money_math
        FROM
            default.final_project_check
        WHERE
            subject == 'Math'
        GROUP BY
            st_id
        ) AS E
    ON
        A.st_id = E.st_id
    )

GROUP BY
    test_grp
"""
Metrics = ph.read_clickhouse(query, connection=connection_default)

Metrics

,test_grp,ARPU,ARPAU,ARPPU,CR_all_to_paying,CR_active_to_paying,CR_active_to_paying_math
0,control,4541.0,10906.0,92333.0,0.05,0.12,0.16
1,pilot,11508.0,35365.0,106094.0,0.11,0.33,0.38


### В дополнение:
В качестве проверки рассчитаем те же метрики на питоне.

Сначала аггрегируем данные по группам:

In [9]:
Data_grp = Data_for_metrics.groupby('test_grp', as_index = False)\
                .aggregate({'all_students':'count',
                            'active_students':'count',
                            'active_students_math':'count',
                            'paying_students':'count',
                            'paying_students_math':'count',
                            'money':'sum'
                           })
Data_grp 

,test_grp,all_students,active_students,active_students_math,paying_students,paying_students_math,money
0,control,305,127,49,15,8,1385000.0
1,pilot,295,96,42,32,16,3395000.0


Теперь посчитаем метрики:

In [10]:
Metrics_2 = Data_grp.assign(ARPU = round(Data_grp.money / Data_grp.all_students,0), 
                            ARPAU = round(Data_grp.money / Data_grp.active_students,0),
                            ARPPU = round(Data_grp.money / Data_grp.paying_students,0),
                            CR_all_to_paying = round(Data_grp.paying_students / Data_grp.all_students,2),
                            CR_active_to_paying = round(Data_grp.paying_students / Data_grp.active_students,2),
                            CR_active_to_paying_math = round(Data_grp.paying_students_math / Data_grp.active_students_math,2),
                           )
Metrics_2 = Metrics_2[['test_grp','ARPU','ARPAU','ARPPU','CR_all_to_paying','CR_active_to_paying','CR_active_to_paying_math']]

Metrics_2

,test_grp,ARPU,ARPAU,ARPPU,CR_all_to_paying,CR_active_to_paying,CR_active_to_paying_math
0,control,4541.0,10906.0,92333.0,0.05,0.12,0.16
1,pilot,11508.0,35365.0,106094.0,0.11,0.33,0.38


Сравним метрики с теми, что посчитали в запросе sql. Расчеты идентичны:

In [11]:
Metrics_2 == Metrics

,test_grp,ARPU,ARPAU,ARPPU,CR_all_to_paying,CR_active_to_paying,CR_active_to_paying_math
0,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True
